# 통신사 고객의 가입 해지 여부 예측 모델

## 데이터
- gender(성별)<br>
    Whether the customer is a male or a female


- SeniorCitizen(고령자 여부)<br>
    Whether the customer is a senior citizen or not (1, 0)


- Partner(배우자 유무)<br>
    Whether the customer has a partner or not (Yes, No)


- Dependents(부양가족 유무)<br>
    Whether the customer has dependents or not (Yes, No)


- tenure(근속월수)<br>
    Number of <strong>months</strong> the customer has stayed with the company


- PhoneService(휴대폰 유무)<br>
    Whether the customer has a phone service or not (Yes, No)


- MultipleLines(휴대폰 2개 이상 여부)<br>
    Whether the customer has multiple lines or not (Yes, No, No phone service)


- InternetService(인터넷 이용 여부 + 종류)<br>
    Customer’s internet service provider (DSL, Fiber optic, No)


- OnlineSecurity(보안 설정 여부)<br>
    Whether the customer has online security or not (Yes, No, No internet service)
    
    
- OnlineBackup(온라인 백업 여부)<br>
    Whether the customer has online backup or not (Yes, No, No internet service)

 
- DeviceProtection(단말기 보안 설정 여부)<br>
    Whether the customer has device protection or not (Yes, No, No internet service)


- TechSupport(기술 지원 여부)<br>
    Whether the customer has tech support or not (Yes, No, No internet service)


- StreamingTV(TV 시청 여부) <br>
    Whether the customer has streaming TV or not (Yes, No, No internet service)


- StreamingMovies(영화 시청 여부) <br>
    Whether the customer has streaming movies or not (Yes, No, No internet service)


- Contract(약정 기간) <br>
    The contract term of the customer (Month-to-month, One year, Two year)


- PaperlessBilling(디지털 청구서 수신 여부) <br>
    Whether the customer has paperless billing or not (Yes, No)


- PaymentMethod(결제 방식)<br>
    The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))


- MonthlyCharges(월 지불 금액)  <br>
    The amount charged to the customer monthly


- TotalCharges(지불할 총 금액) <br>
    The total amount charged to the customer


- Churn (가입 해지 여부)<br>
    Whether the customer churned or not (Yes or No)

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 13.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        main_df = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
main_df.info()

In [ ]:
main_df.duplicated()

In [ ]:
main_df.head()

------------------------------
object 타입이 많다. <br>
타입 변환이 필요해보인다.<br><br>



In [ ]:
main_df.nunique()

----------------
nunique값을 보니 Yes, No 형식이 제대로 맞춰져있나보다. (ex) YeS,YES,no,no phone service 이런식으로 되어 있는게 없다)

In [ ]:
main_df.Churn.value_counts()

In [ ]:
plt.title("Churn rate")
plt.pie(main_df.Churn.value_counts(), labels= main_df.Churn.value_counts().keys(), autopct = "%.2f", colors= ['hotpink', 'lightpink'])
plt.show()

-------
데이터 불균형이 확인되었다. EDA 후 리샘플링을 수행해야겠다.

<br><br><br>
# 전처리
- 타입 변환
- 결측값 처리

## 총지불금액(TotalCharges) -> float

In [ ]:
main_df['TotalCharges'] = main_df['TotalCharges'].replace(' ','0').astype(float) 


In [ ]:
main_df.describe()

In [ ]:
main_df.loc[main_df.TotalCharges == 0]

-----------------
TotalCharges가 누락된 고객명단 -> contract를 참고해서 예측값을 넣자

## Contract -> int

In [ ]:
dic = {'Month-to-month':1, 'One year':12, 'Two year': 24} 
for c in main_df.Contract.unique():
    main_df.Contract.replace(c,dic[c], inplace = True)

## TotalCharges 누락값 계산 : MonthlyCharges * Contract

In [ ]:
#TotalCharges가 0인 고객 예측값    
main_df['TotalCharges'].replace(0.0,main_df['MonthlyCharges']*main_df['Contract'], inplace = True)

In [ ]:
main_df.loc[main_df.TotalCharges == 0]

-------------------
이제 모든 고객의 TotalCharges가 채워졌다.

In [ ]:
main_df.TotalCharges.describe()

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(main_df.TotalCharges)
plt.xlabel("TotalCharges")
plt.title("Distribution of TotalCharges")
plt.show()

In [ ]:
main_df.TotalCharges.nunique()

-----------
그런데 값의 분포가 너무 넓어서 binning을 수행해야할 것 같다.

In [ ]:
tmp_min = main_df.TotalCharges.min() #최솟값
tmp_max = main_df.TotalCharges.max() #최댓값
tmp_mean = main_df.TotalCharges.mean() #평균
bins = [tmp_min-1,tmp_mean/7,tmp_mean/2,tmp_mean,tmp_max/2,tmp_max] #최소~평균/7,평균/7~평균/2,평균/2 ~평균,평균~최대/2,최대/2~최대 
bins

In [ ]:
main_df['TotalCharges_bin'] = pd.cut(main_df.TotalCharges, bins, labels = list(range(len(bins)-1)))
main_df['TotalCharges_bin'].head()

In [ ]:
main_df.TotalCharges_bin.value_counts()

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(main_df.TotalCharges_bin)
plt.xlabel("TotalCharges_bin")
plt.title("Distribution of TotalCharges_bin")
plt.show()

------------------
5개의 그룹을 생성하여 분포의 범위를 줄였고, 그룹당 구성원 수도 비슷하게 맞추었다. 

## object -> category

In [ ]:
main_df.nunique()

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Encoder = LabelEncoder()
# for c in filter(lambda x: (main_df[x].dtype == 'O' and main_df[x].nunique() <=4 and x!='Churn') or x=='SeniorCitizen', main_df.columns):
#     main_df[c]= Encoder.fit_transform(main_df[c])

In [ ]:
for c in filter(lambda x: (main_df[x].dtype == 'O' and main_df[x].nunique() <=4) or x=='SeniorCitizen', main_df.columns):
    main_df[c] = main_df[c].astype("category")

In [ ]:
main_df.info()

In [ ]:
main_df.describe()

--------------------------------
customerID를 제외한 모든 object타입을 적절한 타입으로 변환시켰다.

<br><br><br>
# EDA
- 각 속성과 Churn의 상관관계 분석
- 속성 간의 상관관계 분석
- 분석을 토대로 구체적인 가설을 세우고 검증하기


**필요에 따라 파생속성 생성*

In [ ]:
#시각화를 위한 color 딕셔너리
color_dic = {0:['#005b6e','#04668c'],1:['#3c6ca7','#786eb7'],2:['#a86bba','#da66ac'],
             3:['#ff6792','#aa6792'],4:['#d8e0bb','#b6cec7'],5:['#ff5050','#ffaa00'],
            6:['#ff99ff','#ff6699',],7:['#e6f2ff','#99ccff'],8:['#ccccff','#cc99ff'],
            9:['#ff9966', '#ff6600']}

## TotalCharges와 Churn의 상관관계

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
plt.title("TotalCharges<mean")
print(main_df.loc[main_df.TotalCharges<tmp_mean].Churn.value_counts())
plt.pie(main_df.loc[main_df.TotalCharges<tmp_mean].Churn.value_counts(),autopct ='%.2f', labels=['No','Yes'])
plt.subplot(2,2,2)
plt.title("TotalCharges>=mean")
print(main_df.loc[main_df.TotalCharges>=tmp_mean].Churn.value_counts())
plt.pie(main_df.loc[main_df.TotalCharges>tmp_mean].Churn.value_counts(),autopct ='%.2f', labels=['No','Yes'], colors=color_dic[0])
plt.subplot(2,2,3)
plt.title("TotalCharges_bin == 0")
print(main_df.loc[main_df.TotalCharges_bin==0].Churn.value_counts())
plt.pie(main_df.loc[main_df.TotalCharges_bin==0].Churn.value_counts(),autopct ='%.2f', labels=['No','Yes'], colors=color_dic[1])
plt.subplot(2,2,4)
plt.title("TotalCharges_bin == 4")
print(main_df.loc[main_df.TotalCharges_bin==4].Churn.value_counts())
plt.pie(main_df.loc[main_df.TotalCharges_bin==4].Churn.value_counts(),autopct ='%.2f', labels=['No','Yes'], colors=color_dic[2])
plt.show()

--------------------
위 결과를 보았을 때 TotalCharges이 높은 고객은 확실히 해지율이 낮다.<br><br>
그리고 그 차이는 양 끝단(TotalCharges가 아주 낮거나 아주 높을 때)에서 더 확연하다.

## Tenure과 Churn의 상관관계

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.title("tenure < tenure.mean")
print(main_df.loc[main_df.tenure<main_df.tenure.mean()].Churn.value_counts())
plt.pie(main_df.loc[main_df.tenure<main_df.tenure.mean()].Churn.value_counts(), labels=['No','Yes'],autopct="%.2f",colors=color_dic[0])
plt.subplot(1,2,2)
plt.title("tenure >= tenure.mean")
print(main_df.loc[main_df.tenure>=main_df.tenure.mean()].Churn.value_counts())
plt.pie(main_df.loc[main_df.tenure>=main_df.tenure.mean()].Churn.value_counts(), labels=['No','Yes'],autopct="%.2f",colors = color_dic[1])

plt.show()

---------------------------
평균을 기준으로 근속월수가 높은 사람이 낮은 사람보다 해지 않을 확률이 훨씬 높다.

## Contract과 Churn의 상관관계

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,3,1)
plt.title("distribution of Contract")
sns.distplot(main_df.Contract)

plt.subplot(2,3,2)
plt.title("Churn")
sns.violinplot(main_df.loc[main_df.Churn=='Yes'].Contract)

plt.subplot(2,3,3)
plt.title("Non Churn")
sns.violinplot(main_df.loc[main_df.Churn=='No'].Contract)

for e,month in enumerate([1,12,24],4):
    plt.subplot(2,3,e)
    plt.title(f"Contract == {month}")
    plt.pie(main_df.loc[main_df.Contract==month].Churn.value_counts(), autopct="%.2f", labels = main_df.loc[main_df.Contract==month].Churn.value_counts().keys(), colors= color_dic[e])

plt.show()

In [ ]:
pd.pivot_table(main_df,index=['Contract'] )

In [ ]:
pd.pivot_table(main_df, index=['Churn'])

--------------
위 결과에 따라 Contract가 높을수록 해지율이 낮아지는 것을 확인할 수 있다.

- Contract가 긴 사람들은 돈을 얼마나 지불하는걸까?

In [ ]:
plt.figure(figsize=(20,7))
plt.subplot(1,2,1)
sns.violinplot(main_df['Contract'], main_df['TotalCharges'])
plt.subplot(1,2,2)
sns.violinplot(main_df['Contract'], main_df['MonthlyCharges'])
plt.show()

---------------
약정이 길면 길수록 MonthlyCharges는 낮은 곳에 많이 분포되어 있고,TotalCharges는 낮은 곳보단 윗쪽에 고루 분포되어 있다.<br>
장기 약정 가입자는 할인 혜택을 많이 받기 때문인 것 같다.

## 디지털 친화도(familiar_with_digital)와 Churn의 상관관계
    - 디지털 친화도 측정 방법: 백업, 보안, 다양한 서비스 이용여부 등 정보를 활용

In [ ]:
main_df.info()

In [ ]:
main_df.iloc[:,6:15]

------------------
위 9개 서비스 이용을 많이할수록 디지털 친화도가 높다고 판단한다.

In [ ]:
'''
DSL,Fiber optic, No Phone service 등 다양한 응답이 있으므로 
공백을 기준으로 split을 수행하고 첫번째 요소가 'No'만 아니면 친화도를 높인다(return 1)
'''

def count_digital(x):
    return  1 if x.split()[0]!='No' else 0

count_dig = [0 for _ in range(len(main_df))]
for e in range(6,15):
    for ind in range(len(main_df)):
        count_dig[ind] += count_digital(main_df.iloc[ind,e])
        
#디지털 친화도 컬럼 추가
main_df['familiar_with_digital'] = count_dig
main_df.head()

In [ ]:
main_df.familiar_with_digital.describe()

In [ ]:
pd.pivot_table(main_df, index=['familiar_with_digital', 'Churn'])

In [ ]:
tmp_df = pd.DataFrame({'familiar_with_digital':[],'Churn rate':[],'Number':[]})
for i in range(1,10):
    target = main_df.loc[main_df.familiar_with_digital == i].Churn.value_counts()
    tmp_df = tmp_df.append({'familiar_with_digital':i,'Churn rate':round(target['Yes']/target.sum()*100,2),'Number':target.sum()},ignore_index=True)

plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.title("churn rate of each familiar_with_digital")
sns.lineplot(x=tmp_df['familiar_with_digital'], y= tmp_df['Churn rate'])
plt.subplot(1,2,2)
plt.title("number of each familiar_with_digital")
sns.lineplot(x=tmp_df['familiar_with_digital'], y= tmp_df['Number'])
# sns.lineplot(x=tmp_df['familiar_with_digital'], y= tmp_df['Churn rate'])
# sns.lineplot(x=tmp_df['familiar_with_digital'], y= tmp_df['Number'])
plt.show()

In [ ]:
tmp_df

------------------
디지털 친화도가 3일때까지는 친화도가 높아질수록 해지할 확률도 높아지는 양의 관계를 보이다가, 4부터는 음의 관계를 보이고 있다.

## 수치형 컬럼들의 상관관계

In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(main_df[['TotalCharges', 'MonthlyCharges', 'familiar_with_digital','tenure','Contract']].corr(), annot=True)
plt.show()

-------------
양의 관계) *0.5이상
- familiar_with_digital 과 TotalCharges
- familiar_with_digital 과 MonthlyCharges
- tenure 과 TotalCharges
- tenure 과 Contract
- MonthlyCharges와 TotalCharges

## gender와 Churn의 상관관계

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.pie(main_df.gender.value_counts(), autopct="%.2f", labels =main_df.gender.value_counts().keys(),colors = color_dic[0])
#남녀 Yes/No 비율
fe = main_df.loc[main_df.gender=='Female']
ma = main_df.loc[main_df.gender=='Male']
plt.subplot(1,3,2)
plt.title("Female's Churn rate")
plt.pie(fe.Churn.value_counts(), labels = fe.Churn.value_counts().keys(), autopct = "%.2f", colors =color_dic[1])
plt.subplot(1,3,3)
plt.title("Male's Churn rate")
plt.pie(ma.Churn.value_counts(), labels = ma.Churn.value_counts().keys(), autopct = "%.2f", colors =color_dic[2])
plt.show()

------------
성비가 비슷하고, 성별로 계약 해지여부 비율도 비슷하다

In [ ]:
pd.pivot_table(main_df, index=['gender','Churn'])

--------------------------
성별에 상관없이 해지를 하지 않은 사람들은 해지한 사람들에 비해 모두 계약기간이 길고, 
<br>
월지불금액과 총지불금액이 모두 높다 그리고 근속월수도 길다. 
<br>
디지털 친화도는 다 비슷해서 큰 의미가 없는 것 같다.

## SeniorCitizen과 Churn의 상관관계

* 우선, SeniorCitizen의 나이 기준은 정해진 것이 없는데, 대부분 약 60대부터 해당된다고 한다.  참고: https://www.seniorliving.org/life/senior-citizen/
 <br><br>->The age of a senior citizen varies according to the source. For example, according to Medicare, a senior is 65 years old or older. However, Social Security benefits are eligible for seniors starting at 62, even though the Social Security Office reports that 67 is the age of retirement. Yet if you are 55 and you visit an Arby’s or McDonald’s you can get a senior discount. By the way, Burger King requires you to be at least 60.

In [ ]:
main_df.loc[main_df.SeniorCitizen == 1]

In [ ]:
pd.pivot_table(main_df, index=['SeniorCitizen'])

In [ ]:
main_df.describe()

In [ ]:
pd.pivot_table(main_df, index=['SeniorCitizen'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import copy
norm_df=copy.deepcopy(main_df)
scaler = MinMaxScaler()
norm_df[['Contract', 'MonthlyCharges','TotalCharges','familiar_with_digital','tenure']]=scaler.fit_transform(main_df[['Contract', 'MonthlyCharges','TotalCharges','familiar_with_digital','tenure']])

In [ ]:
common = pd.pivot_table(norm_df, index=['SeniorCitizen']).transpose()
plt.figure(figsize=(10,5))
plt.plot(common[0])
plt.plot(common[1], label='SeniorCitizen')
plt.xticks(rotation=30)
plt.grid()
plt.legend(loc='upper right')
plt.show()

---------------
SeniorCitizen인 사람들(1)이 그렇지 않은 사람들(0)보다 계약기간은 짧지만 더 높은 금액을 지불하고 있고, 근속월수와 디지털 친화도가 조금 더 높다.<br>
TotalCharges의 차이보다 MonthlyCharges의 차이가 크다.

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.title("Senior Citizen or not")
plt.pie(main_df.SeniorCitizen.value_counts(), autopct = "%.2f",labels = ['True'if k==1 else 'False' for k in main_df.SeniorCitizen.value_counts().keys()], 
        colors =color_dic[1])
plt.subplot(1,3,2)
plt.title("Senior Citizen's Churn rate")
plt.pie(main_df.loc[main_df.SeniorCitizen==1].Churn.value_counts(), autopct = "%.2f",labels = main_df.loc[main_df.SeniorCitizen==1].Churn.value_counts().keys(), 
        colors =color_dic[2])
plt.subplot(1,3,3)
plt.title("Non Senior Citizen's Churn rate")
plt.pie(main_df.loc[main_df.SeniorCitizen==0].Churn.value_counts(), autopct = "%.2f",labels = main_df.loc[main_df.SeniorCitizen==0].Churn.value_counts().keys(), 
        colors =color_dic[3])

plt.show()

---------------
Senior Citizen은 전체 고객의 16.21%에 해당하고, 이들 중 해지한 사람은 41.68%이다.<br>
Senior Citizen이 아닌 사람들 중 해지한 사람은 23.61%이다.


## Partner와 Churn의 상관관계

In [ ]:
main_df.Partner.value_counts()

In [ ]:
pd.pivot_table(main_df, index=['Partner'])

In [ ]:
common = pd.pivot_table(norm_df, index=['Partner']).transpose()
plt.figure(figsize=(10,5))
plt.plot(common['No'],label='No Partner')
plt.plot(common['Yes'])
plt.xticks(rotation=30)
plt.grid()
plt.legend(loc='upper right')
plt.show()

--------
배우자가 있는 고객의 약정기간이 배우자가 없는 고객보다 더 길고, 더 많은 금액을 지불하고 있다. 그리고 디지털 친화도와 근속월수도 더 길다. <br> *뭐든 다 높다.
<br>
*특히 Contract와 TotalCharges 그리고 tenure의 값이 압도적으로 높다.


In [ ]:
pd.pivot_table(main_df,index= ['Partner','Churn'])

----------------------
위 테이블을 보면, <br>
배우자유무에 상관없이, 해지한 고객에 비해 해지하지 않은 고객들이
1. 약정기간이 길고, 
2. 월지불금액은 낮지만 전체지불금액이 높고, 
3. 디지털친화도가 아주 조금 더 낮고, 
4. 근속월수가 길다.
<br><br>

*TotalCharges와 Churn의 상관관계 분석을 통해서는 TotalCharges가 높을 수록 Churn=='No'일 확률이 높다는 것을 알고 있었다. <br>
*다음엔 MonthlyCharges와 Churn의 상관관계를 분석해봐야겠다.
<br>

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.title('Parter Yes or No')
plt.pie(main_df.Partner.value_counts(), autopct= "%.2f", labels = main_df.Partner.value_counts().keys(), colors=color_dic[5])
plt.subplot(1,3,2)
plt.title("Partner: No")
plt.pie(main_df.loc[main_df.Partner == 'No'].Churn.value_counts(),autopct = '%.2f',labels=main_df.loc[main_df.Partner == 'No'].Churn.value_counts().keys(),colors = color_dic[0])
plt.subplot(1,3,3)
plt.title("Partner: Yes")
plt.pie(main_df.loc[main_df.Partner == 'Yes'].Churn.value_counts(),autopct = '%.2f',labels=main_df.loc[main_df.Partner == 'Yes'].Churn.value_counts().keys(),colors = color_dic[2])
plt.show()

--------------------------------
배우자가 있는 사람보다 없는 사람의 해지비율이 더 높다.


## MonthlyCharges와 Churn의 상관관계

In [ ]:
main_df.MonthlyCharges.describe()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Distribution of MonthlyCharges")
sns.distplot(main_df.MonthlyCharges)
plt.show()

--------------------------
MonthlyCharges의 값을 특정 구간으로 나눈 MonthlyCharges_bin이라는 컬럼을 하나 더 만들어서 살펴보자

- MonthlyCharges_bin

In [ ]:
bins = [0]
for i in range(1,11,1):
    bins.append(main_df.MonthlyCharges.quantile(0.1*i))
main_df['MonthlyCharges_bin'] = pd.cut(main_df.MonthlyCharges, bins, labels = list(range(len(bins)-1)))
pd.DataFrame(bins)

In [ ]:
pd.DataFrame(main_df.MonthlyCharges_bin.value_counts())

--------------------
MonthlyCharges값을 백분율 10%단위로 끊어서 MonthlyCharges_bin컬럼을 만들어보았다. 

In [ ]:
pd.pivot_table(main_df, index=['MonthlyCharges_bin', 'Churn'])

In [ ]:
tmp_df = pd.DataFrame({'MonthlyCharges_bin':[],'Churn rate':[]})
for i in range(10):
    target = main_df.loc[main_df.MonthlyCharges_bin == i].Churn.value_counts()
    tmp_df = tmp_df.append({'MonthlyCharges_bin':i,'Churn rate':round(target['Yes']/target.sum()*100,2)},ignore_index=True)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x=tmp_df['MonthlyCharges_bin'], y= tmp_df['Churn rate'])
plt.show()

-----------------
MonthlyCharges_bin의 값과 해지율이 선형적인 관계를 가질 줄 알았다.<br>
하지만 bin이 5-8인경우에 해지율이 가장 높고, bin이 0-1인경우 해지율이 가장 낮았다. 나머지는 20%대로 비슷하다.<br>
해당 컬럼도 충분히 의미 있을 수 있지만, 선형적인 관계를 만들어보고 싶기에 MonthlyCharges_bin2 컬럼을 만들어보기로 했다.

- MonthlyCharges_bin2

In [ ]:
bins = [0,main_df.MonthlyCharges.quantile(.35),main_df.MonthlyCharges.quantile(.65),
        main_df.MonthlyCharges.quantile(1)]
bins

In [ ]:
main_df['MonthlyCharges_bin2'] = pd.cut(main_df.MonthlyCharges, bins, labels = list(range(len(bins)-1)))
main_df.MonthlyCharges_bin2.value_counts()

-------
0,백분율 35%의 값, 65%의 값, 최댓값을 구간의 경계값으로 설정했다. 각 구간에 속한 고객수는 2000대 초반~중반으로 맞추었다.


In [ ]:
pd.pivot_table(main_df, index=['MonthlyCharges_bin2', 'Churn'])

In [ ]:
plt.figure(figsize=(20,15))

for e in range(3):
    plt.subplot(3,4,e+1)
    target =main_df.loc[main_df.MonthlyCharges_bin2 == e].Churn.value_counts() 
    plt.pie(target, autopct="%.2f", labels =target.keys(),colors=color_dic[e])
    plt.title(f"MonthlyCharges_bin2 == {e}")
plt.show()


In [ ]:
pct = {}
for i in range(3):
    target = main_df.loc[main_df.MonthlyCharges_bin2 == i].Churn.value_counts()
    pct[i] = round(target['Yes']/target.sum()*100,2)
plt.figure(figsize=(10,5))
sns.lineplot(x=list(range(3)), y=list(pct.values()))
plt.show()

-----------
이렇게 세구간으로 나누니 bin값이 커질수록 해지율이 높아지는 선형적인 관계를 가지게 되었다.<br>


In [ ]:
main_df['MonthlyCharges_bin2']=main_df.MonthlyCharges_bin2.astype('int')

--------------------------------------------------------
따라서 MonthlyCharges_bin2값의 크기가 의미를 가지므로 category -> int로 형변환을 해주었다.

## Dependents와 Churn의 상관관계

In [ ]:
main_df.Dependents.value_counts()

In [ ]:
pd.pivot_table(main_df, index=['Dependents'])

----------------------
부양가족이 있는 고객들이 없는 고객에 비해 약정기간이 길고, 월 지불 금액이 낮고 전체 지불 금액은 높다. 그리고 근속월수도 더 길다.


***->부양가족이 있는 고객들의 해지율이 낮을 것으로 예상한다***


In [ ]:
target_yes = main_df.loc[main_df.Dependents=='Yes'].Churn.value_counts()
target_yes = main_df.loc[main_df.Dependents=='No'].Churn.value_counts()
plt.figure(figsize=(10,10))
for e,ans in enumerate(['Yes','No']):
    plt.subplot(1,2,e+1)
    target= main_df.loc[main_df.Dependents==ans,'Churn'].value_counts()
    plt.pie(target, autopct='%.2f', labels=target.keys(),colors=color_dic[e])
    plt.title(f"Dependents == {ans}")

plt.show()

-----------------
예상대로 부양가족이 있는 고객들의 해지율이 더 낮다.

## PaymentMethod와 Churn의 상관관계

## SeniorCitizen과 Partner의 상관관계

<br><br><br>
# 모델 적용 및 평가

<br>

## Under Sampling

In [ ]:
#수치화
main_df['Churn']= main_df['Churn'].replace({'Yes':1, 'No':0})
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
for c in main_df.columns:
    main_df[c]= Encoder.fit_transform(main_df[c])

In [ ]:
from imblearn.under_sampling import *

In [ ]:
#sampling 이전
main_df['Churn'].value_counts()

In [ ]:
X_under, Y_under = RandomUnderSampler(random_state=0).fit_sample(main_df[main_df.columns], main_df['Churn'])
Y_under.value_counts()

In [ ]:
X_under

<br>

## Over Sampling

In [ ]:
from imblearn.over_sampling import *

In [ ]:
X_over, Y_over = RandomOverSampler(random_state=0).fit_sample(main_df[main_df.columns], main_df['Churn'])
Y_over.value_counts()

In [ ]:
X_over

In [ ]:
#smote--------------------------------------------
smote = SMOTE(random_state=0)
X_smote, Y_smote = smote.fit_sample(main_df[main_df.columns], main_df['Churn'])

In [ ]:
Y_smote.value_counts()

In [ ]:
X_smote

In [ ]:
plt.subplot(1,3,1)

plt.subplot(1,3,2)
plt.subplot(1,3,3)

<br>

## Feature Selection

In [ ]:
feature_all = main_df.columns.drop(['Churn','customerID'])
feature_all

In [ ]:
#familiar_with_digital이 TotalCharges, MonthlyCharges와 높은 양의 관계를 가지므로 해당 컬럼 하나만 적용 
feature1 = ['Contract','familiar_with_digital', 'tenure'] 
feature2 =  ['Contract','familiar_with_digital', 'tenure','SeniorCitizen'] 
feature3,feature4 = feature2[:],feature2[:]
feature3.append('MonthlyCharges_bin')
feature4.append('MonthlyCharges_bin2')


------------------------------------------
위 분석을 토대로 피처선택

<br>

## Model Building & Results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
lgbm_clf = LGBMClassifier()
rf_clf = RandomForestClassifier()
gb_clf = GradientBoostingClassifier()
dt_clf = DecisionTreeClassifier()
svm_clf = svm.SVC(probability = True)
nb_clf = MultinomialNB()
log_clf = LogisticRegression()

names = ['RF','GB','DT','SVM','NB','LOG','LGBM']
model_dic = {'dt': dt_clf, 'svm':svm_clf, 'nb':nb_clf,'log':log_clf, 'gb':gb_clf, 'rf':rf_clf, 'lgbm':lgbm_clf}
target_dic = {'smote':[X_smote, Y_smote],'main':[main_df,main_df['Churn']],'over':[X_over,Y_over], 'under':[X_under,Y_under]}

#get cross_val_score
def get_score(model, feature, custom_cv,target):
    model = model.lower()
    X,Y = target_dic[target]
    return cross_val_score(model_dic[model],X[feature],Y,cv=custom_cv).mean()

#results visualization
def figure_results(model, feature,target):
    X,Y = target_dic[target]
    X_train, X_test, y_train, y_test = train_test_split(X[feature],Y,test_size=0.3,random_state=0)
    model_dic[model.lower()].fit(X_train,y_train)
    
    predictions = model_dic[model.lower()].predict(X_test)
    probabilities = model_dic[model.lower()].predict_proba(X_test)
    auc_score = roc_auc_score(y_test,predictions)
    
    conf_matrix = confusion_matrix(y_test,predictions)
    fpr,tpr,thresholds = roc_curve(y_test,probabilities[::,1])
    print (f"\n {model.upper()} Classification report : \n",classification_report(y_test,predictions))
    plt.figure(figsize=(20,5))
    plt.subplot(1,2,1)
    plt.subplots_adjust(top=0.8)
    plt.suptitle(f"input:{target}, model:{model}\n\n",fontsize=20)
    plt.title("confusion matrix")
    sns.heatmap(conf_matrix,annot=True,fmt = "d",square = True,
                xticklabels=["not churn","churn"],
                yticklabels=["not churn","churn"],
                linewidths = 2,linecolor = "w",cmap = "Set1")
    plt.ylabel('actual')
    plt.xlabel('predicted')
    
    plt.subplot(1,2,2)
    plt.title(f"roc curve: {round(auc_score,3)}")
    sns.scatterplot(x=fpr,y=tpr)
    sns.lineplot(x=[0,1], y=[0,1])
    plt.ylabel('TPR')
    plt.xlabel('FPR')

- cross-validation

In [ ]:

score_df = pd.DataFrame({'model':[], 'feature':[], 'score':[], 'cv': [],'target':[]})
for n in names[3:]:
    score_df = score_df.append({'model':n,'feature':'2','score':get_score(n,feature2,10,'main'),'cv':10,'target':'main_df'}, ignore_index=True)
    score_df = score_df.append({'model':n,'feature':'2','score':get_score(n,feature2,10,'under'),'cv':10,'target':'under'}, ignore_index=True)
    score_df = score_df.append({'model':n,'feature':'2','score':get_score(n,feature2,10,'over'),'cv':10,'target':'over'}, ignore_index=True)
    score_df = score_df.append({'model':n,'feature':'2','score':get_score(n,feature2,10,'smote'),'cv':10,'target':'smote'},ignore_index=True)
score_df

In [ ]:
score_df2 = pd.DataFrame({'model':[], 'feature':[], 'score':[], 'cv': [],'target':[]})
for n in names[4:]:
    score_df2 = score_df2.append({'model':n,'feature':'all','score':get_score(n,feature_all,10,'main'),'cv':10,'target':'main_df'}, ignore_index=True)
    score_df2 = score_df2.append({'model':n,'feature':'all','score':get_score(n,feature_all,10,'under'),'cv':10,'target':'under'}, ignore_index=True)
    score_df2 = score_df2.append({'model':n,'feature':'all','score':get_score(n,feature_all,10,'over'),'cv':10,'target':'over'}, ignore_index=True)
    score_df2 = score_df2.append({'model':n,'feature':'all','score':get_score(n,feature_all,10,'smote'),'cv':10,'target':'smote'},ignore_index=True)
score_df2

In [ ]:
#결과 합치기
results = pd.concat([score_df,score_df2], axis=0)

In [ ]:
pd.pivot_table(results, index=['target','model'])

---------
샘플링한것 중에서는 smote가 가장 성능이 좋은 것으로 보인다.<br>

- 결과 시각화

In [ ]:
for model in ['RF','LOG','LGBM']:
    figure_results(model, feature_all,'smote')

In [ ]:
for model in ['RF','LOG','LGBM']:
    figure_results(model, feature_all,'main')

In [ ]:
# from sklearn.model_selection import GridSearchCV
# params = {'n_estimators': [i for i in range(10,100,15)], 'max_depth':[i for i in range(3,20,5)],
#          'min_samples_leaf':[i for i in range(5,20,5)], 'min_samples_split':[i for i in range(10,50,10)]}
# grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=6, n_jobs = -1, verbose=3)#n_jobs=-1: 모든 코어 사용,verbose: 로그출력 레벨 조정 
# grid_cv.fit(main_df[feature_all], main_df['Churn'])


In [ ]:
# print("최적 하이퍼 파라미터", grid_cv.best_params_)

In [ ]:

# print("최고 예측 정확도:{:.3f}".format(grid_cv.best_score_))

-----------------------------
# 앞으로 남은 일
<개별 속성과 Churn의 상관관계 분석>
- PaymentMethod와 Churn의 상관관계 분석(개별 속성값 관찰 마무리)<br><br>

<속성 간의 관계 관찰>
- Senior와 Partner의 상관관계 분석
- PaymentMethod와 Total&MonthCharges의 상관관계 분석
- familiar_with_digital개선 (지금은 별 의미 없어 보임)
- 모두 수치화해서 모든 컬럼별 상관관계 분석
- familiar_with_digital에서 Charges랑 관련있는거 빼보기<br><br>
<데이터 불균형 해결>
- 다운/업샘플링 
<br><br>
<모델 최적화>
- Grid search cv
- category 타입 one hot encoding 꼭 해야하나? (한것, 안한것 비교해보기)
https://medium.com/analytics-vidhya/data-cleaning-and-preprocessing-a4b751f4066f
<br><br>
<분석 내용 총정리>
- EDA, 결과에 대한 해석 정리